In [1]:
# AOC Day 10

%load_ext autoreload
%autoreload 2

from aoc.utils import *

In [16]:
files: list[int] = []
gaps: list[int] = []

def restart():
    global files, gaps
    files = []
    gaps = []
    with open("input.txt") as file:
        row = file.read().strip()
        is_file = True
        for c in row:
            is_file = not is_file
            if not is_file:
                files.append(int(c))
                continue
            gaps.append(int(c))

In [20]:
# Part 1:
restart()
index, checksum = 0, 0
file_id = 0
while True:
    current_file, files = files[0], files[1:]
    for _ in range(current_file):
        checksum += file_id * index
        index += 1
    file_id += 1
    if len(files) == 0:
        break

    current_gap, gaps = gaps[0], gaps[1:]
    for _ in range(current_gap):
        checksum += (file_id + len(files) - 1) * index
        files[-1] -= 1
        index += 1
        if files[-1] == 0:
            files = files[:-1]
        if len(files) == 0:
            break
print("Part 1:", checksum)

Part 1: 6301895872542


In [19]:
# Part 2:
restart()
# File ID, Length, Starting Index
located_files: list[tuple[int, int, int]] = []
# Length, Starting Index
located_gaps: list[tuple[int, int]] = []

file_id, index = 0, 0
while True:
    current_file_length, files = files[0], files[1:]
    located_files.append((file_id, current_file_length, index))
    index += current_file_length
    file_id += 1
    if len(files) == 0:
        break

    current_gap, gaps = gaps[0], gaps[1:]
    located_gaps.append((current_gap, index))
    index += current_gap

def insert_into_gap(located_file: tuple[int, int, int], gaps: list[tuple[int, int]]):
    file_id, file_length, file_start_index = located_file
    for i, (gap_length, gap_start_index) in enumerate(gaps):
        if gap_start_index > file_start_index:
            continue
        if gap_length >= file_length:
            moved_file = (file_id, file_length, gap_start_index)
            gaps[i] = (gap_length - file_length, gap_start_index + file_length)
            if gap_length == file_length:
                gaps = gaps[:i] + gaps[i + 1:]
            return moved_file, gaps, True
    return located_file, gaps, False

shifting_file_id = len(located_files) - 1
while shifting_file_id >= 0:
    current_file = located_files[shifting_file_id]
    moved_file, located_gaps, moved = insert_into_gap(current_file, located_gaps)
    if moved:
        located_files[shifting_file_id] = moved_file
    shifting_file_id -= 1

located_files.sort(key=lambda x: x[2])

index, total = 0, 0
for file_id, file_length, file_start_index in located_files:
    index = file_start_index
    for _ in range(file_length):
        total += file_id * index
        index += 1

print("Part 2:", total)

Part 2: 6323761685944
